# 9장. 플러그인 백신 엔진 개발하기

[플러그인 엔진의 주요 함수]
    - init 
        - 플러그인 엔진을 초기화 => 각자의 악성코드 패턴 파일을 로딩, 필요한 메모리 확보 등의 일을 처리
    - uninit 
        - 플러그인 엔진 종료 => 로딩된 악성코드 패턴 및 할당된 메모리를
    - scan : 악성코드 검사
    - disinfect : 악성코드 치료
    - listvirus : 플러그인 엔진이 진단/치료 가능한 악성코드의 리스트 알려줌
    - getinto : 플러그인 엔진의 주요 정보를 알려줌


In [ ]:

import os 

# 각 플러그인 엔진의 기본 클래스
class KavMain:
    # 플러그인 초기화
    def init(self, plugins_path):
        # 진단 / 치료하는 악성코드 이름
        self.virus_name = 'Dummy-Test-File (not a virus)'

        # 악성코드 패턴 등록 - virus.kmd 파일 로딩하는 부분을 init 함수에 설계하면 됨
        self.dummy_pattern = 'Dummy Engine test file - Anggo Anti-Virus Project'
        
        return 0

    # 플러그인 엔진을 종료
    def uninit(self):
        del self.virus_name
        del self.dummy_pattern
        
        return 0

    # 악성코드 검사
    def scan(self, filehandle, filename):
        try : 
            # 파일을 열어 악성코드 패턴만큼 파일에서 읽음
            fp = open(filename, 'rb')
            buf = fp.read(len(self.dummy_pattern))
            fp.close 
            buf = str(buf, 'utf-8')

            # 악성코드 패턴을 비교
            if buf == self.dummy_pattern:
                # 악성코드 값이 같다면 결과 값을 리턴
                return True, self.virus_name, 0
        except IOError:
            pass

        # 악성코드를 발견하지 못했음을 리턴
        return False, '', -1

    # 악성코드 치료
    def disinfect(self, filename, malware_id):
        try:
            # 악성코드 진단 결과에서 받은 ID 값이 0인가?
            if malware_id == 0:
                os.remove(filename)     # 파일 삭제
                return True
        except IOError:
            pass

        return False    # 치료 실패 리턴
    


    # 플러그인 엔진이 진단/치료 가능한 악성코드 리스트를 알려줌
    def listvirus(self):
        vlist = list()  # 리스트형 변수 선언

        vlist.append(self.virus_name)

        return vlist

    # 플러그인 엔진의 주요 정보를 알려줌
    def getinfo(self):
        info = dict()

        info['author'] = 'Kei Choi'
        info['version'] = '1.0'
        info['title'] = 'Dummy Scan Engine'
        info['kmd_name'] = 'dummy'

        return info


# 10장. 플러그인 백신 엔진의 암/복호화 도구 만들기

In [ ]:
# RC4 테이블 초기화 의사코드
for i from 0 to 255
    S[i]:=i
end for

j := 0

for i from 0 to 255
    j:= (j + S[i] + key[i mod keylength]) mod 256
    swap values of S[i] and S[j]
end for

In [ ]:
# RC4 암/복호화 의사코드
i := 0
j := 0
while GeneratingOutput:
    i := (i+1)mod 256
    j := (j+S[i]) mod 256
    swap values of S[i] and S[j]
    K := S[(S[i] + S[j]) mod 256]
    output K
end while

In [ ]:
import os
import sys
import k2kmdfile

if __name__ == '__main__':
    #---------------
    # 인자값 체크
    #---------------
    if len(sys.argv) != 2:
        print('Usage : kmake.py [python source]')
        exit()
    
    k2kmdfile.make(sys.argv[1], True)

In [ ]:
import hashlib
import os
import py_compile
import random
import shutil
import struct
import zlib
import k2rc4
import k2timelib


# rsa 개인키를 이용해서 주어진 파일을 암호화하여 KMD 파일 생성
def make(src_fname, debug=False):
    #---------------
    # 2) 암호화 대상 파일을 컴파일 또는 복사해서 준비
    #---------------
    fname = src_fname

    if fname.split('.')[1] == 'py':
        py_compile.compile(fname, fname+'c', None, True)        # 컴파일
        pyc_name = fname+'c'                                    # 컴파일 이후 파일명
    else:           # 파이썬 파일이 아닐 경우 확장자를 pyc로 하여 복사
        pyc_name = fname.split('.')[0]+'.pyc'
        shutil.copy(fname, pyc_name)
    

    # KMD 파일 생성
    # 헤더 : 시그니처(KAVM) + 예약영역
    kmd_data = b'KAVM'
    ret_date = k2timelib.get_now_date()
    ret_time = k2timelib.get_now_time()

    val_date = struct.pack('<H', ret_date)
    val_time = struct.pack('<H', ret_time)

    reserved_buf = val_date + val_time + (b'\x00'*28)

    kmd_data += reserved_buf

    # 본문 : 개인키로 암호화한 RC4 키 + RC4로 암호화한 파일
    random.seed()

    while 1:
        tmp_kmd_data = b''
        
        # RC4 알고리즘에 사용할 128bit 랜덤키 생성
        key = random.randbytes(16)
        tmp_kmd_data += key

        # 생성된 pyc 파일 압축하기
        buf1 = open(pyc_name, 'rb').read()
        buf2 = zlib.compress(buf1)

        e_rc4 = k2rc4.RC4()
        e_rc4.set_key(key)

        # 압축된 pyc 파일 이미지를 RC4로 암호화 
        buf3 = e_rc4.crypt(buf2)

        e_rc4 = k2rc4.RC4()
        e_rc4.set_key(key)

        # 암호화한 압축된 pyc 파일 이미지를 복호화하여 결과가 같은지를 확인
        if e_rc4.crypt(buf3) != buf2:
            continue

        tmp_kmd_data += buf3


        # 꼬리 : 개인키로 암호화한 MD5x3
        md5 = hashlib.md5()
        md5hash = kmd_data + tmp_kmd_data
        for i in range(3):
            md5.update(md5hash)
            md5hash = md5.hexdigest()

            md5hash = bytes(md5hash, 'utf-8')

        kmd_data += tmp_kmd_data + md5hash
        break
    
    # KMD 파일 생성
    ext = fname.find('.')
    kmd_name = fname[0:ext] + '.kmd'

    try:
        if kmd_data:
            open(kmd_name, 'wb').write(kmd_data)
            os.remove(pyc_name)

            if debug:
                print('Success : %-13s -> %s' % (fname, kmd_name))
            return True
        else:
            raise IOError
        
    except IOError:
        if debug:
            print('Fail : %s' % (fname))
        return False
    